# 依存関係逆転の原則（**D**ependency Inversion Principle：DIP）

**上位のモジュールは、下位のモジュールに依存してはならない。どちらのモジュールも抽象に依存すべきである。**

**抽象は実装の詳細に依存してはならない。実装の詳細が抽象に依存すべきである。**

**上位のモジュール**

⇒　人間に近い処理を担当（例：ビジネスのルール、アプリケーションのユースケース、など）

⇒　アプリケーションの**目的**（なぜ存在するのか？）を体現する




**下位のモジュール**

⇒　コンピュータに近い処理を担当（例：データベースアクセス、アルゴリズム、外部との通信、など）

⇒　アプリケーションの目的達成の**手段**（どう実現するのか？）を体現する

次のコードでは、

上位モジュールである注文処理（process_order）が

*   ユーザーの有効性チェック
*   在庫チェック
*   支払い処理
*   在庫更新処理

といった、データベースアクセスや外部サービスの利用などを伴う、

下位モジュールから構成されている

In [ ]:
# eコマースの注文処理

# 下位モジュール
def validate_user(user_name: str) -> bool:
    # ユーザーの有効性をチェック
    pass

# 下位モジュール
def check_stock(product_name: str) -> bool:
    # 在庫をチェック
    pass

# 下位モジュール
def make_payment(user_name: str, product_name: str) -> bool:
    # 支払いを処理
    pass

# 下位モジュール
def update_stock(product_name: str) -> None:
    # 在庫を更新
    pass

# 上位モジュール (下位のモジュールに依存している) → 下位のモジュールを使用しなげればならないから
# 上位モジュールが下位モジュールの変更などによって大きな影響を受ける
def process_order(user_name: str, product_name: str) -> str:
    if not validate_user(user_name):
        return '無効なユーザーです。'
    if not check_stock(product_name):
        return '在庫切れです。'
    if not make_payment(user_name, product_name):
        return '購入額が足りません。'

    update_stock(product_name)
    return '注文処理が完了しました。'

## 構造化プログラミングと依存の方向性

オブジェクト指向プログラミング以前の、構造化プログラミングでは、

大きな機能を、複数の小さな機能に分解するような設計を行っていたため、

上位モジュールが下位モジュールに依存しやすかった

上記のような、注文処理を実装する場合、

*   ユーザーの有効性チェック
*   在庫チェック
*   支払い処理
*   在庫更新処理

といった機能に分解して、それぞれの機能を分担して開発する

⇒　このやり方だと、上位モジュールが下位モジュールに依存することになる

⇒　**下位モジュールの変更に、上位モジュールが影響を受ける**構造


しかし、上位モジュールが下位モジュールの変更に影響を受けるのは、おかしな話

⇒　言うなれば、手段の変更によって、目的を変えなければいけなくなるようなもの

⇒　レストランの店員「電子マネー決済なら(手段: 下位モジュール)、ハンバーグは注文できないですね(目的: 上位モジュール)」




構造化プログラミングの時代から、依存の方向性を「**逆転**」させて、

常に手段（下位モジュール）が目的（上位モジュール）に依存するべき



⇒　これを達成するための原則が、DIP

上位のモジュールは、下位のモジュールに依存してはならない。どちらのモジュールも**抽象に依存すべき**である。

抽象は実装の詳細に依存してはならない。**実装の詳細が抽象に依存すべき**である。

## 抽象への依存とは

抽象への依存とは、Pythonにおいては、**抽象クラスへの依存**の意味

インターフェースのある言語では、インターフェースへの依存も含む



次のコードにおいて、

*   具象クラス(サブクラス・継承先)（EmailNotification, SMSNotification）
*   notify関数

はどちらも、抽象クラス（AbstractNotification）に依存している

In [ ]:
from abc import ABC, abstractmethod

# 上位のモジュール (理由: インターフェース自体には具体的な実装は記述されないから) (何らかの通知を送るという情報だけ)
class AbstractNotification(ABC): # 抽象クラス・インターフェースの役割
    @abstractmethod
    def send(self, user_id: int) -> None:
        pass

# 下位のモジュール (理由: 具体的にどのような通知を送信するのかを記述するから) (メールで通知を送信するという具体的な実装を記述する)
class EmailNotification(AbstractNotification): # 具象クラス
    def send(self, user_id: int) -> None:
        # メールで通知を送信
        print('メール')

# 下位のモジュール (理由: 具体的にどのような通知を送信するのかを記述するから) (SMSで通知を送信するという具体的な実装を記述する)
class SMSNotification(AbstractNotification): # 具象クラス
    def send(self, user_id: int) -> None:
        # SMSで通知を送信
        print('SMS')

# 上位モジュール (理由: 通知を送るという最終的な目的を表すから) (最終的に決定された何らかの通知を送るという目的を表す)
def notify(user_id: int, notification: AbstractNotification):
    notification.send(user_id)

モジュールAがモジュールBに依存していることを「A → B」で表すと、

```
notify (上位モジュール) → AbstractNotification (notifyのインターフェース) (B: 上位モジュール)
                                ↑
                        EmailNotification, SMSNotification (A: 下位モジュール)

```



この例では、

*   上位モジュール：notify関数（とnotify関数が使うインターフェース(AbstractNotification)）
*   下位モジュール：具象クラス(EmailNotification, SMSNotification)

なので、下位モジュールが上位モジュールに依存していると言える

⇒　依存関係「逆転」

### 抽象に依存するメリット・デメリット

抽象に依存すると、

*   下位モジュール(手段)の変更が、上位モジュール(目的)に及ばなくなる
*   具象クラスよりも、抽象クラスの方が変更されづらい
*   具象クラスを切り替えられるようになる（OCPの実現、LSPは前提）

ので、変更に強いコードになる



また、具象クラスに依存しないので、

*   共同開発がしやすくなる
*   テストがしやすくなる

というメリットもある

下記のコードでは、

Orderクラスが、AbstractPaymentクラスに依存している。

そのため、AbastractPaymentを継承した具象クラスを好きに切り替えることができて、手段を変更しても目的に影響が及ばない

開発中のテストの側面で見たとき、継承した具体的な決済のクラスがいくつか定義されるが、この決済のクラスの実装っていうのは結構時間がかる場合がある。

もし仮にこのオーダークラスが具体的な決済のクラスに依存してしまうとなると、具体的な決済のクラスの完成には時間がかかり、不完全な未完成な部品に依存しなければならなくなってしまう。

そうすると、このオーダークラスの機能が正常に実装されているかのテストがしづらくなってしまう。

なぜなら、Orderクラスの振る舞いに何か間違いがあった時、それがOrderクラスの実装の間違いから来ているのか、もしくは未完成な具体的な決済のクラスから来ているのかっていうのが判断しづらくなってしまう。

だから抽象への依存を使ってこのような問題を解決することができる

In [ ]:
from abc import ABC, abstractmethod

class AbstractPayment(ABC):
    @abstractmethod
    def pay(self, amount: int) -> bool:
        pass

# 支払い機能のモック、テスト用としても機能する (具象クラス: 手段・下位モジュール)
class MockPayment(AbstractPayment):
    def pay(self, amount: int) -> bool:
        return True
    
# # 具象クラス (手段・下位モジュール)
# # テストをする時にまだ実装されていない(開発中)
# class CreditCardPayment(AbstractPayment):
#     def pay(self, amount: int) -> bool:
#         return False

# # 具象クラス (手段・下位モジュール)
# # テストをする時にまだ実装されていない(開発中)
# class CashPayment(AbstractPayment):
#     def pay(self, amount: int) -> bool:
#         return True

# 具体的な支払い処理の実装の完成を待たずとも、クライアントの実装と動作確認を行える (上位モジュール)
class Order:
    def __init__(self, payment: AbstractPayment) -> None:
        self._payment = payment

抽象クラスを定義する必要があるので、コードが増えるのがデメリットと言えるが、

その他の原則と同じく、正しく適用できれば、基本的にはメリットが上回る

## Dependency Injection（依存性の注入、DI）パターン

DIPと関係の深いデザインパターンとして、

**Dependency Injection（依存性の注入、DI）パターン**がある

その名の通り、依存関係をモジュールの外部から注入するパターン

依存関係を注入することで、動的に依存関係を変更することができる


DIパターンを使った手法の手順 (メソッドで注入するパターン)

1. 抽象クラスを定義する (AbstractNotification)

2. 抽象クラスを継承する具象クラスを定義する (EmailNotification, SMSNotification)

3. DIパターンを使用するための器となる関数(抽象クラスに依存する・抽象クラスを使用する)を定義する。器となる関数の中には、手段として使用する具象クラスを注入するために、具象クラスが継承した同じ抽象クラスを引数として受取り、関数内で使用できるようにする (notify関数の引数の"notification: AbstractNotification" と メソッドの引数の"self.notification.send(user_id)"の"notification"の部分)

4. 関数の呼び出し時に、具象クラスへの依存を注入する(器となる関数に具象クラスのインスタンスを引数として渡す) (notify関数の呼び出し時にEmailNotificationもしくはSMSNotificationを渡す)


In [ ]:
# DIパターンのメソッドで注入するパターン

from abc import ABC, abstractmethod

# 抽象クラス
class AbstractNotification(ABC):
    @abstractmethod
    def send(self, user_id: int) -> None:
        pass

# 具象クラス
class EmailNotification(AbstractNotification):
    def send(self, user_id: int) -> None:
        # メールで通知を送信
        print('メール')

# 具象クラス
class SMSNotification(AbstractNotification):
    def send(self, user_id: int) -> None:
        # SMSで通知を送信
        print('SMS')

# 抽象クラスに依存する(抽象クラスを使用する)関数 (DIパターンを使用するための器となる関数)
# "notify"関数の定義時にはどの具象クラスに依存するかはまだ決まっていない ("send"メソッドを呼び出すときに具体的な具象クラスを注入する)
def notify(user_id: int, notification: AbstractNotification):
    notification.send(user_id)


notification1 = EmailNotification()
notification2 = SMSNotification()

# 関数の呼び出し時に具象クラスへの依存を注入する (引数に入れる = 注入する)
notify(1, notification1) # メールで通知を送信 (EmailNotificationを注入)
notify(2, notification2) # SMSで通知を送信 (SMSNotificationを注入)

メール
SMS


モジュールの定義時には、抽象に依存しておいて（DIP）、

実行時に具象への依存をモジュールに注入する（DI）ことで、

依存関係を動的に決定することができる

⇒　OCPが実現できる（ただし、LSPは前提）

**ちょっとクイズ**

次のコードは、DIPに違反しています

DIPを満たすようにした上で、

DIパターンを使って、FastSwimmingとSlowSwimmingを

Fishオブジェクトの生成時に切り替えられるようにしてください

In [ ]:
# 元のコード
class FastSwimming:
    def swim(self) -> str:
        return '速く泳ぎます'

class SlowSwimming:
    def swim(self) -> str:
        return 'ゆっくり泳ぎます'

class Fish:
    def __init__(self) -> None:
        self.swimming_behaviour = FastSwimming()

    def swim(self) -> None:
        print(f'魚が{self.swimming_behaviour.swim()}')

解答の一例としては、次のようなコードになる

DIパターンを使った手法の手順 (コンストラクタで注入するパターン)

1. 抽象クラスを定義する (SwimmingBehaviour)

2. 抽象クラスを継承する具象クラスを定義する (FastSwimming, SlowSwimming)

3. DIパターンを使用するための器となるクラス(抽象クラスに依存する・抽象クラスを使用する)を定義し、そのクラスのコンストラクタで手段として使用する具象クラスが継承したものと同じ抽象クラスを引数に受取り、DIパターンを使用するための器となるクラスのインスタンス化時に、注入するクラスを同時にインスタンス化する

	```
	def __init__(self, swimming_type: SwimmingBehaviour) -> None:
		self.swimming_behaviour = swimming_type
	```

4. 抽象クラスに依存する(抽象クラスを使用する)関数を定義する。DIパターンを使用するための器となるクラスのコンストラクタ内でインスタンス化した抽象クラスをメソッド内で使用する (Fishクラスの__init__メソッドの引数の"swimming_type: SwimmingBehaviour")	

5. DIパターンを使用するための器となるクラスのインスタンス化時に、引数に具象クラスを注入する(具象クラスの引数を渡す) (Fishクラスのインスタンス化時にFastSwimmingもしくはSlowSwimmingを渡す)

6. DIパターンを使用するための器となるクラスが抽象クラスに依存する(抽象クラスを使用する)関数を呼び出す (fast_fish.swim()やslow_fish.swim())


In [ ]:
# DIパターンのコンストラクタで注入するパターン

from abc import ABC, abstractmethod

# 抽象クラス
class SwimmingBehaviour(ABC):
    @abstractmethod
    def swim(self) -> str:
        pass

# 具象クラス
class FastSwimming(SwimmingBehaviour):
    def swim(self) -> str:
        return '速く泳ぎます'

# 具象クラス
class SlowSwimming(SwimmingBehaviour):
    def swim(self) -> str:
        return 'ゆっくり泳ぎます'

# 抽象クラスに依存する(抽象クラスを使用する)クラス (DIパターンを使用するための器となるクラス)
class Fish:
    # コンストラクタでDIパターンを使用して、抽象クラスに依存する
    def __init__(self, swimming_type: SwimmingBehaviour) -> None:
        self.swimming_behaviour = swimming_type

    def swim(self) -> None:
        print(f'魚が{self.swimming_behaviour.swim()}')

# Fishオブジェクトの初期化時に、依存性を注入する
fast_fish = Fish(FastSwimming())
fast_fish.swim()

slow_fish = Fish(SlowSwimming())
slow_fish.swim()

魚が速く泳ぎます
魚がゆっくり泳ぎます


依存性を注入する方法としては、

*   コンストラクタ (魚の例のときのコード)
*   メソッドの引数 (notify関数のときのコード)

などがあるが、コンストラクタから注入されることが一番多い

より詳しくは、[Dependency Injection Principles, Practices, and Patterns](https://hiramatsuu.com/archives/1433)を参照

## DIPの主な適用場所

**データベースアクセスや、外部サービスの利用などの、**

**技術的な詳細を扱う場合には、基本的にDIPを適用する**


ユーザーの新規作成を例にして、

データベースアクセスを行ういくつかの方法について見てみましょう

In [ ]:
#　最悪の方法（SQLをコードにベタ書き）
class User:
    def __init__(self, name: str):
        self.name = name

class UserApplicationService:
    # ユーザーの新規作成を行うメソッド
    def register_user(self, name: str) -> None:
        user = User(name)

        # データベースに接続する処理
        query = f"INSERT INTO users (name) VALUES ('{user.name}')"
        # クエリの検証・実行、データベースの開放処理

# 使用例
user_application_service = UserApplicationService()
user_application_service.register_user('ユーザー')

上記のコードは、次のような問題点がある

*   register_userメソッドの責務は、ユーザーの登録のはずなのに、データベース操作の責務も持ってしまっている
*   データベースが変わったときに、register_userメソッドを変更する必要がある
*   データベースと密結合しているため、register_userメソッドのユニットテストが難しい


In [ ]:
# クラスを適用した方法
class User:
    def __init__(self, name: str):
        self.name = name

# Repositoryパターン (手段・下位モジュール)
class SQLiteUserRepository:
    def add(self, user: User) -> None:
        # データベースに接続する処理
        query = f"INSERT INTO users (name) VALUES ('{user.name}')"
        # クエリの検証・実行、データベースの開放処理

# ユーザーのユースケースを表現するクラス (目的・上位モジュール)
class UserApplicationService:
    def __init__(self):
        self.repository = SQLiteUserRepository() # 抽象に依存している (手段・下位モジュール)

    def register_user(self, name: str) -> None:
        user = User(name)
        self.repository.add(user)

# 使用例
user_application_service = UserApplicationService()
user_application_service.register_user('ユーザー')

上記のコードは、前述のコードと比較して

*   データベース操作をSQLiteUserRepositoryクラスで抽象化した
*   その結果、データベース操作の責務がSQLiteUserRepositoryクラスに分割された




これは、ドメイン駆動設計の**Repositoryパターン**と呼ばれる設計パターン

⇒　データベース操作用のクラスを用意することで、データベース操作の詳細を隠蔽する

一方で、次のような問題点もある
*   UserApplicationServiceという上位モジュールが、SQLiteUserRepositoryという下位モジュールに依存している
*   データベースを動的に切り替えることができず、ユニットテストがしづらい

上記の問題点を解決するために、DIPを適用する

DIパターンを使った手法の手順 (コンストラクタで注入してメソッドで使用するパターン)

1. 抽象クラスを定義する (UserRepository)

2. 抽象クラスを継承する具象クラスを定義する (SQLiteUserRepository, InMemoryUserRepository)

3. DIパターンを使用するための器となるクラス(抽象クラスに依存する・抽象クラスを使用する)を定義し、そのクラスのコンストラクタで手段として使用する具象クラスが継承したものと同じ抽象クラスを引数に受取り、DIパターンを使用するための器となるクラスのインスタンス化時に、注入するクラスを同時にインスタンス化する

	```
	def __init__(self, repository: UserRepository): # 抽象に依存、DIパターン
    self.repository = repository
	```

4. 抽象クラスに依存する(抽象クラスを使用する)関数を定義する。DIパターンを使用するための器となるクラスのコンストラクタ内でインスタンス化した抽象クラスをメソッド内で使用する

	```
	def register_user(self, name: str) -> None:
		user = User(name)
		self.repository.add(user)
	```

5. DIパターンを使用するための器となるクラスのインスタンス化時に、引数に具象クラスを注入する(具象クラスの引数を渡す) (UserApplicationServiceクラスのインスタンス化時に引数として、SQLiteUserRepositoryもしくはInMemoryUserRepositoryのインスタンスを渡す)

6. DIパターンを使用するための器となるクラスが抽象クラスに依存する(抽象クラスを使用する)関数を呼び出す (user_application_service.register_user('ユーザー') もしくは user_application_service.register_user('ユーザー'))

In [ ]:
#　DIPを適用した方法

from abc import ABC, abstractmethod

class User:
    def __init__(self, name: str):
        self.name = name


# リポジトリの抽象を用意する
# 抽象クラス
class UserRepository(ABC):
    @abstractmethod
    def add(self, user: User) -> None:
        pass


# SQLiteのリポジトリ
# 具象クラス
class SQLiteUserRepository(UserRepository):
    def add(self, user: User) -> None:
        # データベースに接続する処理
        query = f"INSERT INTO users (name) VALUES ('{user.name}')"
        # クエリの検証・実行、データベースの開放処理


# インメモリのリポジトリ
# 具象クラス
class InMemoryUserRepository(UserRepository):
    def __init__(self):
        self.users = []

    def add(self, user: User) -> None:
        self.users.append(user)


# ユーザーのユースケースを表現するクラス
# 抽象に依存する(抽象クラスを使用する)クラス (DIパターンを使用するための器となるクラス)
class UserApplicationService:
    def __init__(self, repository: UserRepository): # 抽象に依存、DIパターン
        self.repository = repository

    # 抽象に依存する(抽象クラスを使用する)関数
    def register_user(self, name: str) -> None:
        user = User(name)
        self.repository.add(user)

# 使用例
sqlite_repository = SQLiteUserRepository()
user_application_service = UserApplicationService(sqlite_repository)
user_application_service.register_user('ユーザー')

in_memory_repository = InMemoryUserRepository()
user_application_service = UserApplicationService(in_memory_repository)
user_application_service.register_user('ユーザー')

上記のコードでは、
*   DIPを満たすようになった
*   リポジトリをSQLiteとインメモリで切り替えられるようになる

データベースや外部サービスへの依存は、

テストケースを作りづらくするため、

DIPを適用してテスト用の具象クラスと切り替えられた方が便利

より詳しくは、[単体テストの考え方/使い方](https://hiramatsuu.com/archives/1433)を参照

## 他の原則との関係性

DIPとLSPを守ることで、OCPを実現できる


## DIPのまとめ

*   依存関係逆転の原則（DIP）は、
  
  「上位のモジュールは、下位のモジュールに依存してはならない。どちらのモジュールも抽象に依存すべきである。
  
    抽象は実装の詳細に依存してはならない。実装の詳細が抽象に依存すべきである。」という原則
*   上位のモジュールは、アプリケーションの「目的」を体現し、下位のモジュールはアプリケーションの「手段」を体現している
*   上位モジュールが下位モジュールに依存すると、上位モジュールが下位モジュールの変更の影響を受けるようになってしまう
*   言うなれば、手段の変更によって、目的を変えなければいけない、というおかしな構造になってしまう
*   構造化プログラミングでは、上位モジュールが下位モジュールに依存する傾向があったが、この方向性を「逆転」するのがDIP
*   抽象に依存することで、変更に強くなり、テストがしやすくなり、共同開発がしやすくなる
*   Dependency Injection（依存性の注入、DI）パターンを使って、

    依存関係をモジュールの外部から注入することで、動的に依存関係を変更することができる
*   データベースアクセスや、外部サービスの利用などの、技術的な詳細を扱う場合には、基本的にDIPを適用する
*   DIPとLSPを守ることで、OCPを実現できる


## DIPの演習問題

①DIPとはどのような原則でしょうか

これまでに学んだことを、できる限り多く思い出してみましょう

②以下の対立する2つの要素において、

下位のモジュールにあたるのは、それぞれどちらでしょうか


*   在庫を管理するクラス vs データベースアクセスを行うクラス
*   センサーデータを管理するクラス vs センサーデータに基づいてパターン認識を行うクラス
*   通知を送るクラス vs プッシュ通知を送るクラス

③DIPの適用によって、共同開発やユニットテストがしやすくなるのは、なぜでしょうか

④次のコードは、DIPに違反しているでしょうか

違反している場合は、その理由を述べた上で、

DIPを満たすようにコードを改善してください


In [ ]:
class GasolineEngine:
    def start(self) -> None:
        print('ガソリンエンジンが始動しました')

class NormalTires:
    def __init__(self) -> None:
        self._pressure: int = 100

    def is_inflated(self) -> bool:
        if self._pressure >= 1:
            return True
        else:
            print('タイヤに空気を入れてください')
            return False

    def use_air(self) -> None:
        self._pressure -= 1

class Car:
    def __init__(self) -> None:
        self.engine = GasolineEngine()
        self.tires = NormalTires()

    def start(self) -> None:
        if self.tires.is_inflated():
            self.engine.start()
            self.tires.use_air()
            print('車が発進しました')

⑤次のコードは、DIPに違反しているでしょうか

違反している場合は、その理由を述べた上で、

DIPを満たすようにコードを改善してください


In [ ]:
#　ユーザーを表現するクラス
class User:
    def __init__(self, id: int, name: str):
        self.id = id
        self.name = name

# RDBへのアクセスを担当するクラス
class RDBUserRepository:
    def save(self, user: User) -> None:
        print(f'Save {user.name} to the RDB')  # 疑似的なRDBへの保存処理

    def get(self, id: int) -> User:
        print(f'Get user from the RDB by id {id}')  # 疑似的なRDBからの取得処理
        return User(id, 'user_name')

# ユーザーに関するユースケースを実現するクラス
class UserApplicationService:
    def __init__(self):
        self.user_repository = RDBUserRepository()

    def create(self, id: int, name: str):
        user = User(id, name)
        self.user_repository.save(user)